In [2]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [3]:
# If working in Colab, mount your drive
from google.colab import drive
drive.mount('/content/drive')

# Direct to your project folder.
%cd /content/drive/Othercomputers/Computer1/dl-robocop

Mounted at /content/drive
/content/drive/Othercomputers/Computer1/dl-robocop


In [4]:
# Installing requirements
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86

In [5]:
from datasets import load_dataset
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import evaluate
#import gensim
import transformers
import nltk
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
#import jsonlines

In [6]:
data_path = 'data/'
dataset = pd.read_csv(data_path + 'paradetox.tsv', sep='\t')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# BART MODEL (STUDENT)
model_name = "facebook/bart-large"
student_tokenizer = AutoTokenizer.from_pretrained(model_name)
student_tokenizer.pad_token_id = student_tokenizer.eos_token_id
student_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

#model.load_state_dict(torch.load('bart_test.pth'))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [25]:
# PARADETOX MODEL (TEACHER)
from transformers import BartForConditionalGeneration, AutoTokenizer
base_model_name = 'facebook/bart-base'
model_name = 'SkolkovoInstitute/bart-base-detox'
teacher_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
teacher_model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [8]:
# POLITENESS JUDGE
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# load tokenizer and model weights
classifier_tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
toxicity_clasifier_model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
from torch.utils.data import TensorDataset, DataLoader

# Define the maximum length and other parameters
max_length = 38

# Generate the input_ids and input_mask for each sample
input_ids = [torch.tensor(student_tokenizer.encode(d, max_length=max_length, padding='max_length')) for d in dataset['toxic']]
detoxified_ids = [torch.tensor(student_tokenizer.encode(d, max_length=max_length, padding='max_length')) for d in dataset['neutral1']]
input_mask = [torch.tensor([1 if token != student_tokenizer.pad_token_id else 0 for token in tokens]) for tokens in input_ids]

# Convert lists to tensors
input_ids_tensor = torch.stack(input_ids)
detoxified_ids_tensor = torch.stack(detoxified_ids)
input_mask_tensor = torch.stack(input_mask)

# Create a TensorDataset
tensor_dataset = TensorDataset(input_ids_tensor, detoxified_ids_tensor, input_mask_tensor)

In [10]:
from torch.utils.data.dataset import random_split

# Define the size of the training set
train_size = int(0.95 * len(tensor_dataset))
test_size = len(tensor_dataset) - train_size

# Split the dataset into training and test sets
train_dataset, test_dataset = random_split(tensor_dataset, [train_size, test_size])

In [11]:
from torch.utils.data import TensorDataset, DataLoader
batch_size = 2
lr = 3e-5
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = 16, shuffle = False)
optimizer = torch.optim.Adam(student_model.parameters(), lr=lr)
num_epochs = 10

In [12]:
def evaluate(model):
  model.eval()  # Set the model to evaluation mode
  tot_loss = 0.0

  # Iterate through the test dataloader
  for batch in test_dataloader:
      input_ids, detoxified_ids, input_mask = batch

      # Move tensors to the device
      input_ids = input_ids.to(device)
      detoxified_ids = detoxified_ids.to(device)
      input_mask = input_mask.to(device)

      # Disable gradient computation
      with torch.no_grad():
          # Forward pass
          model_outputs = model(input_ids=input_ids, attention_mask=input_mask, labels=detoxified_ids)
          loss = model_outputs.loss

          # Accumulate total loss
          tot_loss += loss.item()

  # Calculate average loss over all batches
  average_loss = tot_loss / len(test_dataloader)

  # Print average loss for the test dataset
  print(f"Test Average Loss: {average_loss:.4f}")
  model.train()

In [17]:
def calculate_toxicity_score(output_sequence):
  judge_input = classifier_tokenizer.encode(output_sequence, return_tensors='pt')
  politeness_distribution = torch.softmax(toxicity_clasifier_model(judge_input).logits, dim = -1)[0]
  toxicity_indicator = torch.argmax(politeness_distribution).item()
  toxicity_score = (((-1) ** toxicity_indicator * politeness_distribution[toxicity_indicator]) + 1)/2
  return toxicity_score

In [14]:
def policy_sampling(model, input_ids, mask, max_seq_length):
    decoder_inputs = {'input_ids': input_ids, 'attention_mask': mask}
    output_ids = torch.tensor([])
    sequence_probas = torch.tensor([])
    past_key_values = None
    for t in range(max_seq_length):
        model_outputs = model(**decoder_inputs, use_cache=True, past_key_values=past_key_values)
        past_key_values = model_outputs['past_key_values']
        logits = model_outputs.logits[:, -1, :]
        probabilities = torch.softmax(logits, dim=-1)

        # Sample from the modified distribution
        next_tokens = torch.multinomial(probabilities, 1)
        probabilities = torch.gather(input=probabilities, dim=1, index= next_tokens)

        decoder_inputs['input_ids'] = next_tokens
        decoder_inputs['attention_mask'] = torch.ones_like(next_tokens).unsqueeze(1)

        output_ids = torch.cat([output_ids, next_tokens], dim=1)
        sequence_probas =  torch.cat([sequence_probas, probabilities], dim = 1)

    output_ids = output_ids.to(torch.int)

    return output_ids, sequence_probas



In [20]:
def paradetox_train(alpha):
  for epoch in range(num_epochs):
    student_model.train()
    total_loss = 0.0
    for i,batch in enumerate(train_dataloader):
      input_ids, detoxified_ids, input_mask = batch

      input_ids = input_ids.to(device)
      detoxified_ids = detoxified_ids.to(device)
      input_mask = input_mask.to(device)

      g_student_outputs = student_model(input_ids=input_ids, attention_mask=input_mask, labels=detoxified_ids)
      s_student_outputs, s_student_probas = policy_sampling(student_model, input_ids, input_mask, 40)

      g_output_sequences = np.array([student_tokenizer.decode(g_student_outputs.logits.argmax(dim = -1)[i]) for i in range(batch_size)])
      s_output_sequences = np.array([student_tokenizer.decode(s_student_outputs[i]) for i in range(batch_size)])

      g_toxicity_scores = torch.tensor([calculate_toxicity_score(g_output_sequences[i]) for i in range(batch_size)])
      s_toxicity_scores = torch.tensor([calculate_toxicity_score(s_output_sequences[i]) for i in range(batch_size)])

      ml_loss = g_student_outputs.loss
      rewards = (g_toxicity_scores - s_toxicity_scores) # + BLEU score + whatever

      sum_probas = torch.sum(torch.log(s_student_probas), dim = -1)
      rl_loss = torch.mean(rewards * sum_probas)
      full_loss = alpha * rl_loss + (1- alpha) * ml_loss
      print(f'{rl_loss = }')
      print(f'{full_loss = }')

      full_loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      total_loss += full_loss.item()
      # Print average loss for the epoch
      #print(f"Train Loss: {loss.item():.4f}")
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}, Average train Loss: {average_loss:.4f}")
    print(f'Evaluating on test set...')
    evaluate(student_model)
    print('\n')


In [ ]:
paradetox_train(alpha = 0.1)

rl_loss = tensor(1.6757e-06, grad_fn=<MeanBackward0>)
full_loss = tensor(6.7927, grad_fn=<AddBackward0>)
rl_loss = tensor(-0.0005, grad_fn=<MeanBackward0>)
full_loss = tensor(7.4057, grad_fn=<AddBackward0>)
rl_loss = tensor(1.1113e-05, grad_fn=<MeanBackward0>)
full_loss = tensor(4.9918, grad_fn=<AddBackward0>)
rl_loss = tensor(0.0009, grad_fn=<MeanBackward0>)
full_loss = tensor(2.9864, grad_fn=<AddBackward0>)
rl_loss = tensor(0.0019, grad_fn=<MeanBackward0>)
full_loss = tensor(4.6768, grad_fn=<AddBackward0>)


In [116]:
def knowledge_expansion_training():
  for epoch in range(num_epochs):
    student_model.train()
    teacher_model.eval()
    total_loss = 0.0
    for i,batch in enumerate(train_dataloader):
      input_ids, detoxified_ids, input_mask = batch
      input_ids = input_ids.to(device)
      detoxified_ids = detoxified_ids.to(device)
      input_mask = input_mask.to(device)

      teacher_model_outputs = teacher_model(input_ids=input_ids, attention_mask=input_mask, labels=detoxified_ids)
      student_model_outputs = student_model(input_ids=input_ids, attention_mask=input_mask, labels=detoxified_ids)


      loss = model_outputs.loss

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      total_loss += loss.item()
      # Print average loss for the epoch
      #print(f"Train Loss: {loss.item():.4f}")
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}, Average train Loss: {average_loss:.4f}")
    print(f'Evaluating on test set...')
    evaluate()
    print('\n')

In [ ]:
evaluate()

Test Average Loss: 1.5222


In [ ]:
prompt = "Rewrite the following toxic input into non-toxic version. Let's break the input down step by step to rewrite the non-toxic version. You should first think about the expanation of why the input text is toxic. Then generate the detoxic output. You must preserve the original meaning as much as possible.\nInput: "

for (idx,batch) in enumerate(test_dataloader):
  #input = prompt+input+"\n"
  model.eval()
  input_ids, detoxified_ids, mask = batch

  input_ids = input_ids.to(device)
  detoxified_ids = detoxified_ids.to(device)
  mask = mask.to(device)

  outputs = model.generate(input_ids = input_ids, attention_mask = mask, max_new_tokens = 50, do_sample=False)
  print('Toxic sentence: ', ' '.join([tokenizer.decode(i, skip_special_tokens = True) for i in input_ids[0]]))
  print(f'Detoxified sentence:\n {tokenizer.decode(outputs[0], skip_special_tokens = True)}')
  print('Gold: ', ' '.join([tokenizer.decode(i, skip_special_tokens = True) for i in detoxified_ids[0]]))
  print()
  if idx == 10:
    break

Toxic sentence:   what  do  you  expect  from  the  social  worker  in  chief ,  k athy  w yn ne ,  the  economic  im bec ile .              
Detoxified sentence:
 What do you expect from the social worker in chief, Kathy wynne.
Gold:   what  do  you  expect  from  the  social  worker  in  chief ,  k athy  w yn ne .                    

Toxic sentence:   they  are  shameless  staggering  hypoc rites .                              
Detoxified sentence:
 They are hypocrites.
Gold:   They  are  hypoc rites                                 

Toxic sentence:   shit  chill in  whats  up  with  you  bro  how  practice  and  school  go in                       
Detoxified sentence:
 What's up with you bro how practice and school going
Gold:   ch illin  whats  up  with  you  bro  how  practice  and  school  going                         

Toxic sentence:   f ucking  mor ons  the  lot  of  you  if  you  think  this  is nt  something  they  would  do  or  at  least  consider .              
Detoxi

In [ ]:
torch.save(model, 'bart_test.pth')